# OpenMC Tallies

In this tutorial, we will learn how to:

  - Understand application of filters and scores to create tallies
  - Apply tallies to an OpenMC simulation
  - Extract information from OpenMC statepoint files
  - Understand tally units and normalization
  - Plot tally results

In [ ]:
import openmc

In this section, we'll be looking at how to extract custom information from an OpenMC simulation in what is known as a "tally." A tally accumulates statistical information during the simulation about particles when they eneter regions of phase space specified on the tally. The limits of these regions are set by "filters" applied to the tally. Scores and nuclides can also be applied to tallies to indicate what type of information is kept about the particle (e.g. reaction types, flux, heat, etc.).

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \underbrace{\psi (\mathbf{r}, \mathbf{\Omega}, E)}_{\text{angular flux}}
$$

where filters set the limits of the integrals and the scoring function is convolved with particle information (e.g. reaction type, current material, etc.). For example, if you wanted to calculate the fission reaction rate caused by fast neutrons in cell 3, your tally becomes

$$ 
 X = \int_\text{cell 3} d\mathbf{r} \int_{4\pi} d\mathbf{\Omega} \int_{1 MeV}^{20 MeV}
    dE \ \ \Sigma_f(\mathbf{r}, \mathbf{\Omega},
    E) \psi (\mathbf{r}, \mathbf{\Omega}, E)
$$

<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>

## Pincell Model

First we'll need a model to examine. OpenMC has a few basic models that we can use to look at tally setup. The function below generates a 2-D PWR pincell model with reflective boundary conditions on the X-Y planes. This function provides an `openmc.Model` object, which ties together materials, geometry, tallies, and settings in a single Python object with a full problem description.

In [ ]:
model = openmc.examples.pwr_pin_cell()

To get a better idea of what this model looks like, we'll start by generating a plot and examining the materials used.

In [ ]:
model.geometry.root_universe.plot()

In [ ]:
model.geometry.get_all_materials()

If we look at the tallies object on our pincell model, we'll see there aren't currently any custom tallies applied.

In [ ]:
model.tallies

In this exercise we'll be adding tallies to perform a few different tasks:


  **1. Determine the energy and heat produced per fission** \
  **2. Plot the flux spectrum of the pincell** \
  **3. Plot reaction types based on material**
  
To do this we'll use a variety of different filters applied to different tallies. 

<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>

## Energy released per fission

To compute the energy released per fission, we will use two different scores -- the `kappa-fission` score, which tallies the recoverable energy release from fission, and the `fission` score, which tallies the fission rate. The energy released per fission, averaged over all fission events, is simply the `kappa-fission` score divided by the `fission` score.

Because we want this information talllied throughout the model, a "global" tally, no filters need to be applied.

In [ ]:
fission_tally = openmc.Tally()
fission_tally.scores = ['fission', 'kappa-fission']
print(fission_tally)

In [ ]:
model.tallies = openmc.Tallies([fission_tally])


After adjusting the default settings for number of particles and batches on the model we'll run it and examine the data.

In [ ]:
model.settings.batches = 50
model.settings.inactive = 10
model.settings.particles = 10000

In [ ]:
statepoint = model.run()

If we list our current directory, we see that several new files have been created as a result of this run: `summary.h5`, `tallies.out`, and `statepoint.50.h5`. The summary file contains information about the simulation's setup (geometry, materials, meshes, etc.) in an HDF5 format. The `tallies.out` file contains a text output of all user-specified tallies for the simulation.

In [ ]:
!cat tallies.out

This can be useful to quickly look at simple tally results, but isn't a great format to post-process simulation data. For that we'll look to the statepoint file. The statepoint file contains information about simulation results including tally specifications and data. The location of this statepoint file was provided to us by the `model.run()` command.

To extract information from the statepoint file we'll create an `openmc.StatePoint` object. The `statepoint.get_tally` function will search for tallies by scores, filters, nuclides, ids, and return the closest match. Exact matches can be specified as well.

In [ ]:
with openmc.StatePoint('statepoint.50.h5') as sp:
    my_tally = sp.get_tally(id=fission_tally.id)

If we print the tally objects returned, we see that they indeed match the tally specification we generated above.

In [ ]:
print(my_tally)

In [ ]:
# material, nuclide, score
my_tally.shape

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>



To compute the energy released per fission event, we can simply take the tallied energy released per fission and divide it by the fission rate.

In [ ]:
fission_rate = my_tally.get_values(scores=['fission']).flatten()[0]
kappa_fission_rate = my_tally.get_values(scores=['kappa-fission']).flatten()[0]

ev_per_fission = kappa_fission_rate / fission_rate
mev_per_fission = ev_per_fission * 1e-6

print('MeV per fission: ', mev_per_fission)

For a water reactor with U235 as the only fissioning isotope this is about what we would expect: ~193 MeV! 

When dealing with tallies, we must remember that every output of a Monte Carlo simulation is uncertain -- it is associated with a mean and a standard deviation. Whenever you present results from a Monte Carlo simulation, you should ALWAYS present the mean value AND its standard deviation, or else your results are kind of meaningless (and you may be eviscerated!). To obtain the standard deviation associated with our MeV/fission estimation, we can use the Python [uncertainties](https://pythonhosted.org/uncertainties/) module, while also using the `value='std_dev'` option when fetching the tally values to get the standard deviations.

In [ ]:
fission_rate_std_dev = my_tally.get_values(scores=['fission'], value='std_dev').flatten()[0]
kappa_fission_rate_std_dev = my_tally.get_values(scores=['kappa-fission'],value='std_dev').flatten()[0]

from uncertainties import ufloat
f = ufloat(fission_rate, fission_rate_std_dev)

kf = ufloat(kappa_fission_rate, kappa_fission_rate_std_dev) 

print('MeV per fission: {:.6f}'.format((kf / f * 1e-6)))

As with most values coming out of an MC code, these values are per source-particle. In this case these units cancel out, but this will not be the case in our next example.

## Plot the neutron flux spectrum


Plotting a neutron flux spectrum is a very useful way to understand the physical processes happening to neutrons - the energy at which they exist is determined by scattering reactions (to lower energies) as well as their birth distribution (such as from fission or fusion). It is often an engineer's objective to control the energies at which neutrons are predominantly at in their system in order to encourage reactions at different energy spectra.

To plot the neutron flux spectrum, we'll be applying a tally with an energy filter and a score. OpenMC's data module contains different group structures. For this problem we'll use the CASMO-70 group structure. An energy filter can easily be created from a pre-defined group structure in OpenMC as follows:

In [ ]:
energy_filter = openmc.EnergyFilter.from_group_structure('CASMO-70')
len(energy_filter.bins)

In [ ]:
spectrum_tally = openmc.Tally()
spectrum_tally.filters = [energy_filter]
spectrum_tally.scores = ['flux']
print(spectrum_tally)

Now we'll apply this tally and re-run the problem

In [ ]:
model.tallies = [spectrum_tally]
statepoint = model.run()

In [ ]:
with openmc.StatePoint('statepoint.50.h5') as sp:
    tally = sp.get_tally(id=spectrum_tally.id)

In [ ]:
spectrum = tally.get_values(value='mean').flatten()
print(len(spectrum))

In [ ]:
spectrum[:5]

Now to plot the spectrum, we will plot the neutron flux per unit lethargy (a common way to visualize neutron flux).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

unit_lethargy = energy_filter.lethargy_bin_width
print(unit_lethargy)

In [ ]:
print(energy_filter.bins)

In [ ]:
plt.step(np.unique(energy_filter.bins)[:-1], spectrum / unit_lethargy)
plt.xscale('log')
plt.xlabel('Energy (eV)')
plt.ylabel('Flux per unit lethargy')
plt.show()

## Normalizing Tallies

Note that the units of flux in the above plot are in $\frac{particle-cm}{source-particle}$. As is the case with many values tallied by Monte Carlo codes, the value of the flux does not account for volume and is in terms of the number of source particles emitted. To generate this same plot in terms of absolute flux units ($\frac{particle}{cm^{2}-s}$) we'll need to normalize this tally by:

  - the volume of the region the tally covers
  - the number of source particle emitted

In this case, the volume of the region is the volume of the entire pincell. Because we're working with a 2-D model, we'll get units that give us the flux per unit length of the pincell in the axial direction. For simplicity, we'll assume that our pincell is 1 cm in height to make life easier. OpenMC's geometry object allows for computation of a bounding box for geometries like this that will help make this more straight forward.

In order to obtain the volumes of the cells, we can use a stochastic volume calculation. Or, in this case, we can obtain the volumes from a bounding box of the geometry.

In [ ]:
lower_left, upper_right = model.geometry.bounding_box
print(lower_left, upper_right)

In [ ]:
lower_left[-1] = 0.0
upper_right[-1] = 1.0

volume = np.prod(upper_right - lower_left)
print(volume)

Determining the number of source particles per second is more complicated, however. This means computing the eV/source particle due to fission. To get the source rate, we'll need the following pieces of information:

  1. the total power produced in the tally region (known a priori)
  2. the heat produced by fission power, per source particle
  
 To get this information we'll need to construct another tally to get additional information from the simulation.

In [ ]:
source_tally = openmc.Tally()
source_tally.scores = ['kappa-fission']
model.tallies = [source_tally]

In [ ]:
statepoint = model.run(output=False)

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    source = sp.get_tally(id=source_tally.id)
    
heating = source.get_values().flatten()[0]

The combination of the following tally values and power provide us with the source normalization needed as follows:


$$ \text{neutron source} [\frac{n}{s}] = \text{power} [\frac{J}{s}] \times \frac{1}{1.6\times 10^{-19}} [\frac{eV}{J}] \times \frac{1}{\text{heat per fission} [\frac{eV}{source}]} $$ 

In [ ]:
J_to_eV = 1 / 1.6e-19
power = 200 # Watts
neutron_source = power * J_to_eV * (1 / heating)

print(f'Neutron source: {neutron_source:.2e} n/s')

We can now use this information to normalize our flux values and reproduce our plot in more standard units.

In [ ]:
normalized_spectrum = neutron_source * spectrum / volume

In [ ]:
plt.step(np.unique(energy_filter.bins)[:-1], normalized_spectrum / unit_lethargy)
plt.xscale('log')
plt.xlabel('Energy (eV)')
plt.ylabel('Flux per unit lethargy')
plt.show()

## Reaction Types by Material

Looking at the different reaction types by material will require a material filter and the set of reaction types we want to score. For this example, we'll be scoring absorption, scattering and fission in each material. To start, we'll create a material filter.

In [ ]:
model.materials

In [ ]:
material_filter = openmc.MaterialFilter(model.materials)

In [ ]:
material_tally = openmc.Tally()
material_tally.filters = [material_filter]
material_tally.scores = ['absorption', 'scatter', 'fission']

In [ ]:
model.tallies = [material_tally]

In [ ]:
statepoint = model.run()

Now we'll gather information from the statepoint file about each score we applied to the tally. With multiple scores and materials, we'll use a Pandas data frame to view the results in a more coherent manner.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(id=material_tally.id)
    absorption = tally.get_values(scores=['absorption']).flatten()
    scatter = tally.get_values(scores=['scatter']).flatten()
    fission = tally.get_values(scores=['fission']).flatten()
    df = tally.get_pandas_dataframe()

Each score has three values -- one for each material in the model.

In [ ]:
df

First, we'll add a new column to the data frame with normalized results.

In [ ]:
df['normalized-mean (rxn/s)'] = neutron_source * df['mean']
df

We'll add a new entry in the dataframe for our material names to make plotting easier.

In [ ]:
# get all materials from the geometry
materials = model.geometry.get_all_materials()

type(materials)

In [ ]:
# set names based on matching material IDs
for mat_id, material in materials.items():
    df.loc[df['material'] == mat_id, 'mat_name'] = material.name
df

In [ ]:
fission_df = df[df['score'] == 'fission']
fission_df

In [ ]:
fission_df.plot('mat_name', 'normalized-mean (rxn/s)', kind='bar', ylabel='fissions / s')

In [ ]:
scatter_df = df[df['score'] == 'scatter']
scatter_df.plot('mat_name', 'mean', kind='bar', ylabel='scatters / s')

absorption_df = df[df['score'] == 'absorption']
absorption_df.plot('mat_name', 'mean', kind='bar', ylabel='absorptions / s')